In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from random import seed
from random import sample
import sys

In [4]:
seed(1)

In [5]:
sys.path.append('../metrics')

### Efficiency

In [6]:
import efficiency as efficiency

In [7]:
# Data Preparation
# Please note that all datasets are generated randomly. The metric results might be unreasonable.

# Datasets for cost efficiency
label_dataset_columns = {
    "ds_month": [sample(["2023-01-01", "2023-02-01"], 1)[0] for _ in range(1000)],
    "id": [sample(list(range(1000, 10000)), 1)[0] for _ in range(1000)],
    "vertical": [sample(["A", "B"], 1)[0] for _ in range(1000)],
    "use_case": [sample(["uc1", "uc2"], 1)[0] for _ in range(1000)],
    "vendor": [sample(["V1", "V2"], 1)[0] for _ in range(1000)],
    "labeler_type": [sample(["L1", "L2"], 1)[0] for _ in range(1000)],
    "decision_value": [sample(["dv1", "dv2", "not_valid"], 1)[0] for _ in range(1000)],
    "final_decision": [sample(["dv1", "dv2", "not_valid"], 1)[0] for _ in range(1000)],
    "handling_times_sec": [sample(range(1, 1000), 1)[0] for _ in range(1000)],
}
efficiency_label_dataset = pd.DataFrame.from_dict(label_dataset_columns)
cost_dataset_columns = {
    "language": ["english", "english", "english", "spanish", "spanish"],
    "labeler_type": ["L1", "L1", "L2", "L1", "L2"],
    "vendor": ["V1", "V2", "V1", "V1", "V2"],
    "cost_per_second": [0.1, 0.2, 0.3, 0.11, 0.19],
}
efficiency_cost_summary = pd.DataFrame.from_dict(cost_dataset_columns)

In [8]:
efficiency_label_dataset.head()

ds_month    id vertical use_case vendor labeler_type decision_value  \
0  2023-01-01  3295        A      uc2     V1           L2      not_valid   
1  2023-01-01  5201        B      uc1     V2           L1            dv1   
2  2023-02-01  4686        A      uc2     V2           L1            dv2   
3  2023-01-01  2442        B      uc2     V2           L1            dv2   
4  2023-02-01  9828        B      uc1     V2           L1            dv1   

  final_decision  handling_times_sec  
0      not_valid                 404  
1      not_valid                 127  
2      not_valid                 518  
3            dv2                  72  
4            dv2                 101

In [9]:
efficiency_cost_summary

language labeler_type vendor  cost_per_second
0  english           L1     V1             0.10
1  english           L1     V2             0.20
2  english           L2     V1             0.30
3  spanish           L1     V1             0.11
4  spanish           L2     V2             0.19

In [10]:
# merge label dataset and commercial report and get cpd(cost per decision) column
merged_dataset = efficiency.merge_label_commercial_datsets(
    label_data=efficiency_label_dataset,
    commercial_report=efficiency_cost_summary,
    common_cols=["vendor", "labeler_type"],
    cost_per_second="cost_per_second",
    handling_time_second="handling_times_sec",
)
merged_dataset.head()

ds_month    id vertical use_case vendor labeler_type decision_value  \
0  2023-01-01  3295        A      uc2     V1           L2      not_valid   
1  2023-01-01  5201        B      uc1     V2           L1            dv1   
2  2023-02-01  4686        A      uc2     V2           L1            dv2   
3  2023-01-01  2442        B      uc2     V2           L1            dv2   
4  2023-02-01  9828        B      uc1     V2           L1            dv1   

  final_decision  handling_times_sec language  cost_per_second    cpd  
0      not_valid                 404  english              0.3  121.2  
1      not_valid                 127  english              0.2   25.4  
2      not_valid                 518  english              0.2  103.6  
3            dv2                  72  english              0.2   14.4  
4            dv2                 101  english              0.2   20.2

In [11]:
task_level_cost_summary = efficiency.get_task_level_cost_summary(
    dataset=merged_dataset,
    detailed_group_list=["vertical", "vendor", "labeler_type"],
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
    handling_time_second="handling_times_sec",
    cpd="cpd",
)
task_level_cost_summary

cnt_attempted  cnt_converted  conversion_rate  \
vertical vendor labeler_type                                                  
A        V1     L1                      238            170         0.714286   
                L2                      117             78         0.666667   
         V2     L1                      134             89         0.664179   
                L2                      129             84         0.651163   
B        V1     L1                      236            148         0.627119   
                L2                      130             85         0.653846   
         V2     L1                      134             83         0.619403   
                L2                      119             76         0.638655   

                              conversion_rate_sd  ht_attempted  \
vertical vendor labeler_type                                     
A        V1     L1                      0.029283    514.689076   
                L2                      0.043581    510.076923   
         V2     L1                      0.040798    538.402985   
                L2                      0.041962    497.860465   
B        V1     L1                      0.031478    486.822034   
                L2                      0.041725    483.823077   
         V2     L1                      0.041944    485.179104   
                L2                      0.044037    469.512605   

                              ht_attempted_std  cpd_attempted  \
vertical vendor labeler_type                                    
A        V1     L1                   18.975357      54.042353   
                L2                   27.275895     153.023077   
         V2     L1                   25.094493     107.680597   
                L2                   24.830518      94.593488   
B        V1     L1                   17.767958      51.116314   
                L2                   23.476589     145.146923   
         V2     L1                   25.037176      97.035821   
                L2                   24.622588      89.207395   

                              cpd_attempted_std  ht_converted  \
vertical vendor labeler_type                                    
A        V1     L1                     2.001633    720.564706   
                L2                     8.182769    765.115385   
         V2     L1                     5.018899    810.629213   
                L2                     4.717798    764.571429   
B        V1     L1                     1.874458    776.283784   
                L2                     7.042977    739.964706   
         V2     L1                     5.007435    783.301205   
                L2                     4.678292    735.157895   

                              ht_converted_std  cpd_converted  \
vertical vendor labeler_type                                    
A        V1     L1                   26.565500      75.659294   
                L2                   40.913843     229.534615   
         V2     L1                   37.782720     162.125843   
                L2                   38.132581     145.268571   
B        V1     L1                   28.332690      81.509797   
                L2                   35.905371     221.989412   
         V2     L1                   40.421465     156.660241   
                L2                   38.553790     139.680000   

                              cpd_converted_std  
vertical vendor labeler_type                     
A        V1     L1                     2.802286  
                L2                    12.274153  
         V2     L1                     7.556544  
                L2                     7.245190  
B        V1     L1                     2.989001  
                L2                    10.771611  
         V2     L1                     8.084293  
                L2                     7.325220

In [12]:
task_level_cost_summary_by_time = efficiency.get_task_level_cost_summary(
    dataset=merged_dataset,
    detailed_group_list=["vertical", "vendor", "labeler_type"],
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
    handling_time_second="handling_times_sec",
    cpd="cpd",
    time_column="ds_month",
)
task_level_cost_summary_by_time

cnt_attempted  cnt_converted  \
ds_month   vertical vendor labeler_type                                 
2023-01-01 A        V1     L1                      114             86   
                           L2                       51             34   
                    V2     L1                       64             38   
                           L2                       69             46   
           B        V1     L1                      114             74   
                           L2                       64             40   
                    V2     L1                       67             40   
                           L2                       56             35   
2023-02-01 A        V1     L1                      124             84   
                           L2                       66             44   
                    V2     L1                       70             51   
                           L2                       60             38   
           B        V1     L1                      122             74   
                           L2                       66             45   
                    V2     L1                       67             43   
                           L2                       63             41   

                                         conversion_rate  conversion_rate_sd  \
ds_month   vertical vendor labeler_type                                        
2023-01-01 A        V1     L1                   0.754386            0.040315   
                           L2                   0.666667            0.066010   
                    V2     L1                   0.593750            0.061392   
                           L2                   0.666667            0.056750   
           B        V1     L1                   0.649123            0.044698   
                           L2                   0.625000            0.060515   
                    V2     L1                   0.597015            0.059924   
                           L2                   0.625000            0.064694   
2023-02-01 A        V1     L1                   0.677419            0.041979   
                           L2                   0.666667            0.058026   
                    V2     L1                   0.728571            0.053151   
                           L2                   0.633333            0.062212   
           B        V1     L1                   0.606557            0.044228   
                           L2                   0.681818            0.057332   
                    V2     L1                   0.641791            0.058577   
                           L2                   0.650794            0.060061   

                                         ht_attempted  ht_attempted_std  \
ds_month   vertical vendor labeler_type                                   
2023-01-01 A        V1     L1              521.421053         26.924355   
                           L2              513.843137         40.913618   
                    V2     L1              543.171875         34.184076   
                           L2              528.681159         32.972703   
           B        V1     L1              457.175439         24.279410   
                           L2              456.578125         31.956672   
                    V2     L1              474.552239         35.121685   
                           L2              458.285714         35.842277   
2023-02-01 A        V1     L1              508.500000         26.703764   
                           L2              507.166667         36.581127   
                    V2     L1              534.042857         36.472872   
                           L2              462.416667         37.054804   
           B        V1     L1              514.524590         25.565907   
                           L2              510.242424         34.008021   
                    V2     L1              495.805970         35.644592   
          

In [13]:
target_level_cost_summary = efficiency.get_target_level_cost_summary(
    dataset=merged_dataset,
    general_group_list=["vertical", "vendor"],
    final_decision_column="final_decision",
    ids=["id"],
    labeler_type="labeler_type",
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
    handling_time_second="handling_times_sec",
    cpd="cpd",
)
target_level_cost_summary

cnt_task  cnt_target  cnt_final  final_conversion_rate  \
vertical vendor                                                           
A        V1           355         233        151               0.648069   
         V2           263         258        174               0.674419   
B        V1           366         242        163               0.673554   
         V2           253         253        167               0.660079   

                 final_conversion_rate_std   ht_target  ht_target_std  \
vertical vendor                                                         
A        V1                       0.031287  781.866953      35.772384   
         V2                       0.029173  528.565891      18.383411   
B        V1                       0.030143  734.657025      32.531454   
         V2                       0.029780  477.810277      17.613026   

                 cpd_target  cpd_target_std     ht_final  ht_final_std  \
vertical vendor                                                          
A        V1      132.041974        5.418061  1206.456954     55.198447   
         V2      103.223876        3.602477   783.735632     27.258160   
B        V1      127.820455        5.125063  1090.717791     48.298233   
         V2       93.353676        3.454899   723.868263     26.683206   

                  cpd_final  cpd_final_std  
vertical vendor                             
A        V1      203.746887       8.360320  
         V2      153.056092       5.341603  
B        V1      189.770245       7.608989  
         V2      141.428024       5.234068

In [14]:
target_level_cost_summary_by_time = efficiency.get_target_level_cost_summary(
    dataset=merged_dataset,
    general_group_list=["vertical", "vendor"],
    final_decision_column="final_decision",
    ids=["id"],
    labeler_type="labeler_type",
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
    handling_time_second="handling_times_sec",
    cpd="cpd",
    time_column="ds_month",
)
target_level_cost_summary_by_time

cnt_task  cnt_target  cnt_final  \
ds_month   vertical vendor                                    
2023-01-01 A        V1           165         108         70   
                    V2           133         131         91   
           B        V1           178         119         80   
                    V2           123         123         87   
2023-02-01 A        V1           190         127         83   
                    V2           130         129         83   
           B        V1           188         126         86   
                    V2           130         130         80   

                            final_conversion_rate  final_conversion_rate_std  \
ds_month   vertical vendor                                                     
2023-01-01 A        V1                   0.648148                   0.045952   
                    V2                   0.694656                   0.040239   
           B        V1                   0.672269                   0.043029   
                    V2                   0.707317                   0.041025   
2023-02-01 A        V1                   0.653543                   0.042224   
                    V2                   0.643411                   0.042173   
           B        V1                   0.682540                   0.041469   
                    V2                   0.615385                   0.042669   

                             ht_target  ht_target_std  cpd_target  \
ds_month   vertical vendor                                          
2023-01-01 A        V1      793.037037      51.327021  130.585278   
                    V2      543.832061      24.154168  105.981756   
           B        V1      683.521008      42.876153  119.652857   
                    V2      467.146341      25.156160   91.342764   
2023-02-01 A        V1      760.055118      48.630159  131.201339   
                    V2      504.868217      26.251959   98.822868   
           B        V1      765.460317      46.958602  132.490952   
                    V2      487.900000      24.633940   95.256308   

                            cpd_target_std     ht_final  ht_final_std  \
ds_month   vertical vendor                                              
2023-01-01 A        V1            7.483728  1223.542857     79.190262   
                    V2            4.719966   782.879121     34.771385   
           B        V1            6.527625  1016.737500     63.778278   
                    V2            4.937330   660.448276     35.565606   
2023-02-01 A        V1            7.463868  1162.975904     74.410002   
                    V2            5.169254   784.674699     40.801238   
           B        V1            7.078543  1121.488372     68.799812   
                    V2            4.830016   792.837500     40.030152   

                             cpd_final  cpd_final_std  
ds_month   vertical vendor                             
2023-01-01 A        V1      201.474429      11.546323  
                    V2      152.567143       6.794677  
           B        V1      177.983625       9.709842  
                    V2      129.139770       6.980363  
2023-02-01 A        V1      200.753855      11.420617  
                    V2      153.592169       8.034142  
           B        V1      194.114651      10.370888  
                    V2      154.791500       7.848777

In [15]:
survey_cost_summary = efficiency.get_survey_cost_summary(
    dataset=merged_dataset,
    detailed_group_list=["vertical", "vendor", "labeler_type"],
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
)
survey_cost_summary

cnt_attempted  cnt_converted  conversion_rate  \
vertical vendor labeler_type                                                  
A        V1     L1                      238            170         0.714286   
                L2                      117             78         0.666667   
         V2     L1                      134             89         0.664179   
                L2                      129             84         0.651163   
B        V1     L1                      236            148         0.627119   
                L2                      130             85         0.653846   
         V2     L1                      134             83         0.619403   
                L2                      119             76         0.638655   

                              conversion_rate_sd  cost_attempted  \
vertical vendor labeler_type                                       
A        V1     L1                      0.029283               1   
                L2                      0.043581               1   
         V2     L1                      0.040798               1   
                L2                      0.041962               1   
B        V1     L1                      0.031478               1   
                L2                      0.041725               1   
         V2     L1                      0.041944               1   
                L2                      0.044037               1   

                              cost_converted  
vertical vendor labeler_type                  
A        V1     L1                  1.400000  
                L2                  1.500000  
         V2     L1                  1.505618  
                L2                  1.535714  
B        V1     L1                  1.594595  
                L2                  1.529412  
         V2     L1                  1.614458  
                L2                  1.565789

In [16]:
survey_cost_summary_by_time = efficiency.get_survey_cost_summary(
    dataset=merged_dataset,
    detailed_group_list=["vertical",  "vendor", "labeler_type"],
    non_converted_list=["not_valid"],
    decision_value_column="decision_value",
    time_column="ds_month",
)
survey_cost_summary_by_time

cnt_attempted  cnt_converted  \
ds_month   vertical vendor labeler_type                                 
2023-01-01 A        V1     L1                      114             86   
                           L2                       51             34   
                    V2     L1                       64             38   
                           L2                       69             46   
           B        V1     L1                      114             74   
                           L2                       64             40   
                    V2     L1                       67             40   
                           L2                       56             35   
2023-02-01 A        V1     L1                      124             84   
                           L2                       66             44   
                    V2     L1                       70             51   
                           L2                       60             38   
           B        V1     L1                      122             74   
                           L2                       66             45   
                    V2     L1                       67             43   
                           L2                       63             41   

                                         conversion_rate  conversion_rate_sd  \
ds_month   vertical vendor labeler_type                                        
2023-01-01 A        V1     L1                   0.754386            0.040315   
                           L2                   0.666667            0.066010   
                    V2     L1                   0.593750            0.061392   
                           L2                   0.666667            0.056750   
           B        V1     L1                   0.649123            0.044698   
                           L2                   0.625000            0.060515   
                    V2     L1                   0.597015            0.059924   
                           L2                   0.625000            0.064694   
2023-02-01 A        V1     L1                   0.677419            0.041979   
                           L2                   0.666667            0.058026   
                    V2     L1                   0.728571            0.053151   
                           L2                   0.633333            0.062212   
           B        V1     L1                   0.606557            0.044228   
                           L2                   0.681818            0.057332   
                    V2     L1                   0.641791            0.058577   
                           L2                   0.650794            0.060061   

                                         cost_attempted  cost_converted  
ds_month   vertical vendor labeler_type                                  
2023-01-01 A        V1     L1                         1        1.325581  
                           L2                         1        1.500000  
                    V2     L1                         1        1.684211  
                           L2                         1        1.500000  
           B        V1     L1                         1        1.540541  
                           L2                         1        1.600000  
                    V2     L1                         1        1.675000  
                           L2                         1        1.600000  
2023-02-01 A        V1     L1                         1        1.476190  
                           L2                         1        1.500000  
                    V2     L1                         1        1.372549  
                           L2                         1        1.578947  
           B        V1     L1                         1        1.648649  
                           L2                         1        1.466667  
                    V2     L1                         1        1.558140  
                           